See function get_tables at bottom for a single function that returns a pandas dataframe after having scraped the requested data from is-academia. 

All the stuff before that function is explantory text documenting the process of understading the request format to use to the the data we wanted.

In [116]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

base_url = 'http://isa.epfl.ch/imoniteur_ISAP/%21gedpublicreports.htm?ww_i_reportmodel=133685247'

When making a request through the form, the following request is made:
http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_b_list=1&ww_i_reportmodel=133685247&ww_c_langue=&ww_i_reportModelXsl=133685270&zz_x_UNITE_ACAD=Informatique&ww_x_UNITE_ACAD=249847&zz_x_PERIODE_ACAD=&ww_x_PERIODE_ACAD=null&zz_x_PERIODE_PEDAGO=&ww_x_PERIODE_PEDAGO=null&zz_x_HIVERETE=&ww_x_HIVERETE=null&dummy=ok

The params are:  
ww_b_list:1   
ww_i_reportmodel:133685247  
ww_c_langue:  
ww_i_reportModelXsl:133685270  
zz_x_UNITE_ACAD:Informatique  
ww_x_UNITE_ACAD:249847  
zz_x_PERIODE_ACAD:  
ww_x_PERIODE_ACAD:null  
zz_x_PERIODE_PEDAGO:  
ww_x_PERIODE_PEDAGO:null  
zz_x_HIVERETE:  
ww_x_HIVERETE:null  
dummy:ok  

Notice we have specified zz_x_UNITE_ACAD:Informatique i.e. computer science

This gives us a list of all the different student lists for computer science.

This request allows us to select a list and display the students:

http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html?ww_x_GPS=71454914&ww_i_reportModel=133685247&ww_i_reportModelXsl=133685270&ww_x_UNITE_ACAD=249847&ww_x_PERIODE_ACAD=null&ww_x_PERIODE_PEDAGO=null&ww_x_HIVERETE=null

The params are:  
ww_x_GPS:71454914  
ww_i_reportModel:133685247  
ww_i_reportModelXsl:133685270  
ww_x_UNITE_ACAD:249847  
ww_x_PERIODE_ACAD:null  
ww_x_PERIODE_PEDAGO:null  
ww_x_HIVERETE:null  


For the first question we need to filter on Bachelor, the date and the semestre
Here is the request for bachelor semestre 1 2004-2005:
http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html?ww_x_GPS=2225262&ww_i_reportModel=133685247&ww_i_reportModelXsl=133685270&ww_x_UNITE_ACAD=249847&ww_x_PERIODE_ACAD=null&ww_x_PERIODE_PEDAGO=249108&ww_x_HIVERETE=null

ww_x_GPS:2225262  
ww_i_reportModel:133685247  
ww_i_reportModelXsl:133685270  
ww_x_UNITE_ACAD:249847  
ww_x_PERIODE_ACAD:null  
ww_x_PERIODE_PEDAGO:249108  
ww_x_HIVERETE:null  

And for bachelor semestre 6 2004-2005:
http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html?ww_x_GPS=2225150&ww_i_reportModel=133685247&ww_i_reportModelXsl=133685270&ww_x_UNITE_ACAD=249847&ww_x_PERIODE_ACAD=null&ww_x_PERIODE_PEDAGO=942175&ww_x_HIVERETE=null
ww_x_GPS:2225150  
ww_i_reportModel:133685247  
ww_i_reportModelXsl:133685270  
ww_x_UNITE_ACAD:249847  
ww_x_PERIODE_ACAD:null  
ww_x_PERIODE_PEDAGO:942175  
ww_x_HIVERETE:null  

Remark: we seem to have standard values for ww_x_PERIODE_PEDAGO: 
ww_x_UNITE_ACAD:249847 is the code for Informatique
ww_x_PERIODE_PEDAGO is the semester code

In [8]:
bachelor_semestre = {1:249108, 6:942175}
bachelor_semestre

{1: 249108, 6: 942175}

Now lets look at how changing the academic period affects the params:
Bachelor semestre 6 2005-2006:
http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html?ww_x_GPS=2225237&ww_i_reportModel=133685247&ww_i_reportModelXsl=133685270&ww_x_UNITE_ACAD=249847&ww_x_PERIODE_ACAD=null&ww_x_PERIODE_PEDAGO=942175&ww_x_HIVERETE=null

ww_x_GPS:2225237  
ww_i_reportModel:133685247  # Seems to be invariant  
ww_i_reportModelXsl:133685270  
ww_x_UNITE_ACAD:249847  # Section: Computer Science  
ww_x_PERIODE_ACAD:null  
ww_x_PERIODE_PEDAGO:942175  # Semestre  
ww_x_HIVERETE:null  

As expected ww_x_PERIODE_PEDAGO is 942175  
The only value that has changed is ww_x_GPS.

Bachelor Seestre 6 2006-2007:  
http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html?ww_x_GPS=2225324&ww_i_reportModel=133685247&ww_i_reportModelXsl=133685270&ww_x_UNITE_ACAD=249847&ww_x_PERIODE_ACAD=null&ww_x_PERIODE_PEDAGO=942175&ww_x_HIVERETE=null   
ww_x_GPS:2225324  
ww_i_reportModel:133685247  
ww_i_reportModelXsl:133685270  
ww_x_UNITE_ACAD:249847  
ww_x_PERIODE_ACAD:null  
ww_x_PERIODE_PEDAGO:942175  
ww_x_HIVERETE:null  

Again only ww_x_GPS has changed. No obvious pattern, expected that it seems to be an upwards trend as we increase the year. Let's check this. 
As this would be sufficient de filter FROM a specific date.

Bachelor 6 2014-2015:  
ww_x_GPS:1378362238  # OK

BAchelor 6 2016-2017:  
ww_x_GPS:1744378039 # OK the trend seems to hold



Let's look at how to differentiate between bachelor and master
Master 1 2015-2016:
http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html?ww_x_GPS=1897033225&ww_i_reportModel=133685247&ww_i_reportModelXsl=133685270&ww_x_UNITE_ACAD=249847&ww_x_PERIODE_ACAD=null&ww_x_PERIODE_PEDAGO=2230106&ww_x_HIVERETE=null  
ww_x_GPS:1897033225  
ww_i_reportModel:133685247  
ww_i_reportModelXsl:133685270  
ww_x_UNITE_ACAD:249847  
ww_x_PERIODE_ACAD:null  
ww_x_PERIODE_PEDAGO:2230106  
ww_x_HIVERETE:null  

The value that has changed is ww_x_PERIODE_PEDAGO to 2230106  
We can guess that all master 1 students will have this value, whatever the year. Let's check

Master 1 2013 2014  
ww_x_GPS:1378438423  
ww_i_reportModel:133685247  
ww_i_reportModelXsl:133685270  
ww_x_UNITE_ACAD:249847  
ww_x_PERIODE_ACAD:null  
ww_x_PERIODE_PEDAGO:2230106  
ww_x_HIVERETE:null  

Great. The theory holds.


Let's try filtering on the year from their form.
Master 1 2016-2017
http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html?ww_x_GPS=2021044028&ww_i_reportModel=133685247&ww_i_reportModelXsl=133685270&ww_x_UNITE_ACAD=249847&ww_x_PERIODE_ACAD=355925344&ww_x_PERIODE_PEDAGO=2230106&ww_x_HIVERETE=null
ww_x_GPS:2021044028  
ww_i_reportModel:133685247  
ww_i_reportModelXsl:133685270  
ww_x_UNITE_ACAD:249847  
ww_x_PERIODE_ACAD:355925344  
ww_x_PERIODE_PEDAGO:2230106  
ww_x_HIVERETE:null  

Interesting, we have a value for ww_x_PERIODE_ACAD. Let's check this represents the year

Bachelor 6 2016-2017:  
ww_x_GPS:1744378039  
ww_i_reportModel:133685247  
ww_i_reportModelXsl:133685270  
ww_x_UNITE_ACAD:249847  
ww_x_PERIODE_ACAD:355925344  
ww_x_PERIODE_PEDAGO:942175  
ww_x_HIVERETE:null  

Ok great this represents the year. What logic is behind this param ?
Let's go down a year:

BAchelor 6 2015-2016: 
ww_x_GPS:1650772010  
ww_i_reportModel:133685247  
ww_i_reportModelXsl:133685270  
ww_x_UNITE_ACAD:249847  
ww_x_PERIODE_ACAD:213638028  
ww_x_PERIODE_PEDAGO:942175  
ww_x_HIVERETE:null  

Bachelor 6 2014-2015  
ww_x_PERIODE_ACAD:213637922

Ok it looks pretty random, we're just going to get the values for the years we need

In [9]:
# Values for ww_x_PERIODE_ACAD
years = {'2016-2017':355925344,
         '2015-2016':213638028,
         '2014-2015':213637922,
         '2013-2014':213637754,
         '2012-2013':123456101,
         '2011-2012':123455150,
         '2010-2011':39486325,
         '2009-2010':978195,
         '2008-2009':978187,
         '2007-2008':978181 ,     
        }

It also seems there is no obvious where to get ww_x_GPS. Note however that if we request all the lists this param is set to -1. This will do nicely.

In [10]:
def get_data(semestre, year):
    params = {
                'ww_x_GPS':str(-1),  # This gives us all results
                'ww_i_reportModel':133685247,  # Always the same
                'ww_i_reportModelXsl':133685270,  # Always the same
                'ww_x_UNITE_ACAD':249847,  # The section, we also always looking at computer sicence
                'ww_x_PERIODE_ACAD':str(year),  # The school year, ex: 2016-2017
                'ww_x_PERIODE_PEDAGO':str(semestre),  # The semestre, ex Bachelor 1
                'ww_x_HIVERETE':'null'  # Unknown param, always seems to be null
             }
    url = 'http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html?ww_x_GPS={ww_x_GPS}&ww_i_reportModel={ww_i_reportModel}&ww_i_reportModelXsl={ww_i_reportModelXsl}&ww_x_UNITE_ACAD={ww_x_UNITE_ACAD}&ww_x_PERIODE_ACAD={ww_x_PERIODE_ACAD}&ww_x_PERIODE_PEDAGO={ww_x_PERIODE_PEDAGO}&ww_x_HIVERETE={ww_x_HIVERETE}'.format(**params)
    r = requests.get(url)
    return r

In [11]:
# Lets test our function
# assert get_data(bachelor_semestre[1], years['2016-2017']).status_code == 200

In [12]:
# f = open('samplehtml.html','w')
# f.write(get_data(bachelor_semestre[1], years['2016-2017']).text)

Let's start having a look at how we can parse this data:
The data we want is basically in an html table.


In [13]:
data = get_data(bachelor_semestre[1], years['2016-2017'])
soup = BeautifulSoup(data.text, 'html.parser')

In [15]:
soup.table

<table><tr><th colspan="12"><font color="black">Informatique, 2016-2017, Bachelor semestre 1</font>
 (235 ét.)
    </th></tr><tr><th>Civilité</th><th>Nom Prénom</th><th>Orientation Bachelor</th><th>Orientation Master</th><th>Spécialisation</th><th>Filière opt.</th><th>Mineur</th><th>Statut</th><th>Type Echange</th><th>Ecole Echange</th><th>No Sciper</th></tr><tr><td style="white-space:nowrap">Monsieur</td><td style="white-space:nowrap">Abbey Alexandre</td><td style="white-space:nowrap"></td><td style="white-space:nowrap"></td><td style="white-space:nowrap"></td><td style="white-space:nowrap"></td><td style="white-space:nowrap"></td><td style="white-space:nowrap">Présent</td><td style="white-space:nowrap"></td><td style="white-space:nowrap"></td><td>235688</td><td style="white-space:nowrap"></td></tr><tr><td style="white-space:nowrap">Monsieur</td><td style="white-space:nowrap">Ahn Seongho</td><td style="white-space:nowrap"></td><td style="white-space:nowrap"></td><td style="white-space

In [16]:
len(soup.table)

237

In [38]:
table = soup.table

In [87]:
data_list = []
rows = table.find_all('tr')
for i,row in enumerate(rows):
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    cols = cols[:-1]  # There' s an empty column at the end
    data_list.append(cols)
df = pd.DataFrame(data_list[2:],columns=['Civilité','Nom Prénom','Orientation Bachelor','Orientation Master','Spécialisation','Filière opt.','Mineur','Statut','Type Echange','Ecole Echange','No Sciper'])


In [88]:
df.head()



,Civilité,Nom Prénom,Orientation Bachelor,Orientation Master,Spécialisation,Filière opt.,Mineur,Statut,Type Echange,Ecole Echange,No Sciper
0,Monsieur,Abbey Alexandre,,,,,,Présent,,,235688
1,Monsieur,Ahn Seongho,,,,,,Présent,,,274015
2,Madame,Alemanno Sara,,,,,,Présent,,,268410
3,Monsieur,Althaus Luca,,,,,,Présent,,,271464
4,Monsieur,Assi Karim,,,,,,Présent,,,274518


Ok so let's put all of this together in a single function. This function will get the data and return it in a pandas dataframe

In [21]:
def get_table(semestre, year, student_type):
    """
    @param semestre: Int represented the semestre number: for example put 1 for semestre 1
    @param year: String representing the academic year: for example '2016-2017'
    @param student type: String either 'bachelor' or 'master'
    @return: pandas dataframe with corresponding data from Is-Academia
    """
    
    years = {'2016-2017':355925344,
         '2015-2016':213638028,
         '2014-2015':213637922,
         '2013-2014':213637754,
         '2012-2013':123456101,
         '2011-2012':123455150,
         '2010-2011':39486325,
         '2009-2010':978195,
         '2008-2009':978187,
         '2007-2008':978181 ,     
        }
    bachelor_semestre = {1:249108, 2:249114, 3:942155, 4:942163, 5:942120, 6:942175}
    master_semestre = {1:2230106, 2:942192, 3:2230128}
    
    if year in years.keys():
        year_code = years[year]
    else:
        raise Exception('Invalid year selected')
    if semestre in range(1,7):
        if student_type == 'bachelor':
            semestre_code = bachelor_semestre[semestre]
        elif student_type == 'master':
            if semestre in range(1,4):
                semestre_code = master_semestre[semestre]
            else:
                raise Exception('Invalid semestre selected')
        else:
            raise Exception('Invalid student type. Please choose either bachelor or master')
    else:
        raise Exception('Invalid semestre selected')
    
    params = {
                'ww_x_GPS':str(-1),  # This gives us all results
                'ww_i_reportModel':133685247,  # Always the same
                'ww_i_reportModelXsl':133685270,  # Always the same
                'ww_x_UNITE_ACAD':249847,  # The section, we also always looking at computer sicence
                'ww_x_PERIODE_ACAD':str(year_code),  # The school year, ex: 2016-2017
                'ww_x_PERIODE_PEDAGO':str(semestre_code),  # The semestre, ex Bachelor 1
                'ww_x_HIVERETE':'null'  # Unknown param, always seems to be null
             }
    url = 'http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html?ww_x_GPS={ww_x_GPS}&ww_i_reportModel={ww_i_reportModel}&ww_i_reportModelXsl={ww_i_reportModelXsl}&ww_x_UNITE_ACAD={ww_x_UNITE_ACAD}&ww_x_PERIODE_ACAD={ww_x_PERIODE_ACAD}&ww_x_PERIODE_PEDAGO={ww_x_PERIODE_PEDAGO}&ww_x_HIVERETE={ww_x_HIVERETE}'.format(**params)
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    table = soup.table
    data_list = []
    rows = table.find_all('tr')
    for i,row in enumerate(rows):
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        cols = cols[:-1]  # There' s an empty column at the end
        data_list.append(cols)
    columns=['Civilité','Nom Prénom','Orientation Bachelor','Orientation Master','Spécialisation','Filière opt.','Mineur','Statut','Type Echange','Ecole Echange','No Sciper']
    df = pd.DataFrame(data_list[2:],columns=columns)
    return df


In [100]:
# test
get_table(1,'2016-2017', 'master')

,Civilité,Nom Prénom,Orientation Bachelor,Orientation Master,Spécialisation,Filière opt.,Mineur,Statut,Type Echange,Ecole Echange,No Sciper
0,Monsieur,Adams Marc Alain,,,,,,Présent,,,271331
1,Monsieur,Adamsons Valdis,,,,,,Présent,,,273101
2,Monsieur,Agapiou Stylianos,,,,,,Présent,,,267543
3,Monsieur,Aiulfi Loris Sandro,,,,,,Présent,,,202293
4,Monsieur,Alfonso Peterssen Alfonso,,,Foundations of Software,,,Présent,,,228219
5,Monsieur,Angerand Grégoire Georges Jacques,,,,,,Présent,,,212464
6,Monsieur,Anongba Varela Dario Abougnan,,,Software Systems,,,Présent,,,226371
7,Monsieur,Aquil Mohammad Amimul Ihsan,,,Foundations of Software,,,Présent,,,273107
8,Monsieur,Bao Junze,,,,,,Présent,,,266983
9,Monsieur,Barbera Antonino,,,,,,Présent,,,273118


# How long does an EPFL bachelor take?

In [170]:
years = {'2016-2017':355925344,
         '2015-2016':213638028,
         '2014-2015':213637922,
         '2013-2014':213637754,
         '2012-2013':123456101,
         '2011-2012':123455150,
         '2010-2011':39486325,
         '2009-2010':978195,
         '2008-2009':978187,
         '2007-2008':978181 ,     
        }

semesters = (1, 2, 3, 4, 5, 6)

# load all bachelor years and semesters
frames = []
for y in years:
    for sem in semesters:
        f = get_table(sem, y, "bachelor")
        f['year'] = [y]*len(f)
        f['sem'] = [sem]*len(f)
        frames.append(f)
        
frame = pd.concat(frames)

frame = frame.rename(columns={"No Sciper": "sciper", "Nom Prénom":"nom"})
frame.index = [frame.sciper, frame.year]
frame.sort_index()

Civilité                                                nom  \
sciper year                                                                     
NaN    2007-2008      None                                               None   
       2007-2008      None                                               None   
       2007-2008      None                                               None   
       2007-2008      None                                               None   
       2008-2009      None                                               None   
       2008-2009      None                                               None   
       2008-2009      None                                               None   
       2008-2009      None                                               None   
147008 2008-2009  Monsieur                                        Good Xavier   
       2008-2009  Monsieur                                        Good Xavier   
       2009-2010  Monsieur                                        Good Xavier   
       2009-2010  Monsieur                                        Good Xavier   
       2010-2011  Monsieur                                        Good Xavier   
       2010-2011  Monsieur                                        Good Xavier   
153762 2007-2008  Monsieur                                       Conus Johann   
       2007-2008  Monsieur                                       Conus Johann   
154157 2007-2008    Madame                   Andriambololona Riana Miarantsoa   
159516 2007-2008  Monsieur                                        Raja Yanick   
       2007-2008  Monsieur                                        Raja Yanick   
159998 2007-2008    Madame                                        Jesse Julia   
       2007-2008    Madame                                        Jesse Julia   
160213 2007-2008  Monsieur                                 Bécholey Alexandre   
161091 2007-2008    Madame                                   Grivet Ekaterina   
       2007-2008    Madame                                   Grivet Ekaterina   
161127 2007-2008  Monsieur                                      Sahy François   
       2007-2008  Monsieur                                      Sahy François   
161212 2009-2010  Monsieur                                     Tamesna Rachid   
       2009-2010  Monsieur                                     Tamesna Rachid   
161220 2007-2008  Monsieur                                      Tourino Pablo   
       2007-2008  Monsieur                                      Tourino Pablo   
...                    ...                                                ...   
274740 2016-2017  Monsieur                               Durussel Samad Emrys   
274746 2016-2017  Monsieur                                     Beuret Florian   
274748 2016-2017  Monsieur                                    Berla Francesco   
274751 2016-2017    Madame                                  Bochatay Séverine   
274839 2016-2017    Madame                                 Fatton Louise Olga   
274900 2016-2017  Monsieur                                      Laraki Rayane   
274902 2016-2017    Madame                                   Montial Charline   
274918 2016-2017  Monsieur                       Griesser Paul Roland Jacques   
274944 2016-2017  Monsieur                             Gerber Frédéric Julien   
274957 2016-2017    Madame                       Giunta Julie Camille Rosalie   
274960 2016-2017  Monsieur                                      Golubev Maxim   
274999 2016-2017  Monsieur                               Iseli Lucien Michaël   
275001 2016-2017    Madame                                      Idrizi Gonxhe   
275071 2016-2017  Monsieur                                       Krcmar Peter   
275102 2016-2017  Monsieur                                   Friedli Jonathan   
275105 2016-2017  Monsieur                             Fournier Julien Marcel   
275116 2016-2017  Monsieur                       

In [171]:
# group by sciper and count all non zero entries: this gives the enrolled semesters per person
grouped = frame.groupby('sciper').aggregate(np.count_nonzero)['sem']
grouped

sciper
147008    6
153762    2
154157    1
159516    2
159998    2
160213    1
161091    2
161127    2
161212    2
161220    4
161279    4
161634    2
161814    4
161930    6
165562    2
166001    1
166260    2
166344    4
166548    2
166701    4
166746    2
166748    2
166751    2
166889    2
167007    6
167045    2
167313    2
167386    2
167439    2
167447    1
         ..
274740    1
274746    1
274748    1
274751    1
274839    1
274900    1
274902    1
274918    1
274944    1
274957    1
274960    1
274999    1
275001    1
275071    1
275102    1
275105    1
275116    1
275142    1
275159    1
275199    1
275273    1
275313    1
275383    1
275553    1
275608    1
276016    1
276323    1
276507    1
276937    1
277113    1
Name: sem, dtype: int64

In [165]:
# make an average over all semester counts that are greater or equal to 6, if they are lower they haven't finished yet
grouped_sorted = grouped.sort_values(ascending=False)
np.average(grouped_sorted[grouped_sorted >= 6])

7.0

In [174]:
# divide into madame and monsieur
frame.index = [frame.Civilité, frame.sciper, frame.year]
madame = frame.loc['Madame']
monsieur = frame.loc['Monsieur']

,,Civilité,nom,Orientation Bachelor,Orientation Master,Spécialisation,Filière opt.,Mineur,Statut,Type Echange,Ecole Echange,sciper,year,sem
sciper,year,,,,,,,,,,,,,
194197,2009-2010,Monsieur,Abdallah Jad,,,,,,Présent,,,194197,2009-2010,1
186942,2009-2010,Monsieur,Amrani Ismaïl,,,,,,Présent,,,186942,2009-2010,1
194182,2009-2010,Monsieur,Antognini Marco,,,,,,Présent,,,194182,2009-2010,1
186595,2009-2010,Monsieur,Augsburger Damien,,,,,,Présent,,,186595,2009-2010,1
189517,2009-2010,Monsieur,Barben Loïc,,,,,,Présent,,,189517,2009-2010,1
192248,2009-2010,Monsieur,Barbier Issa,,,,,,Présent,,,192248,2009-2010,1
187143,2009-2010,Monsieur,Bennani Kabchi Reda,,,,,,Présent,,,187143,2009-2010,1
195551,2009-2010,Monsieur,Berta Stefano,,,,,,Présent,,,195551,2009-2010,1
193110,2009-2010,Monsieur,Bertrand Thomas,,,,,,Présent,,,193110,2009-2010,1


In [175]:
# average for madame
mad_grouped = madame.groupby('sciper').aggregate(np.count_nonzero)['sem']
mad_grouped_sorted = mad_grouped.sort_values(ascending=False)
np.average(mad_grouped_sorted[mad_grouped_sorted >= 6])

6.78125

In [176]:
# average for monsieur
mon_grouped = monsieur.groupby('sciper').aggregate(np.count_nonzero)['sem']
mon_grouped_sorted = mon_grouped.sort_values(ascending=False)
np.average(mon_grouped_sorted[mon_grouped_sorted >= 6])

7.0159817351598175